# Tugas Praktikum

## Import Library

In [2]:
import pandas as pd
import datetime as dt

## Load Dataset

In [3]:
file_customers = r'D:\Kuliah\Semester2\Artificial_Intelligence\semua_kode\jobsheet_05\tugas_praktikum\sample_data\sample_data\customers.csv'
file_gasstations = r'D:\Kuliah\Semester2\Artificial_Intelligence\semua_kode\jobsheet_05\tugas_praktikum\sample_data\sample_data\gasstations.csv'
file_products = r'D:\Kuliah\Semester2\Artificial_Intelligence\semua_kode\jobsheet_05\tugas_praktikum\sample_data\sample_data\products.csv'
file_transactions = r'D:\Kuliah\Semester2\Artificial_Intelligence\semua_kode\jobsheet_05\tugas_praktikum\sample_data\sample_data\transactions_1k.csv'
file_yearmonth = r'D:\Kuliah\Semester2\Artificial_Intelligence\semua_kode\jobsheet_05\tugas_praktikum\sample_data\sample_data\yearmonth.csv'

## Preview Dataset

In [4]:
df_customers = pd.read_csv(file_customers)
df_customers.head()

,CustomerID,Segment,Currency
0,3,SME,EUR
1,5,LAM,EUR
2,6,SME,EUR
3,7,LAM,EUR
4,9,SME,EUR


In [5]:
df_gasstations = pd.read_csv(file_gasstations)
df_gasstations.head()

,GasStationID,ChainID,Country,Segment
0,44,13,CZE,Value for money
1,45,6,CZE,Premium
2,46,23,CZE,Other
3,47,33,CZE,Premium
4,48,4,CZE,Premium


In [6]:
df_products = pd.read_csv(file_products)
df_products.head()

,ProductID,Description
0,1,Rucní zadání
1,2,Nafta
2,3,Special
3,4,Super
4,5,Natural


In [7]:
df_transactions = pd.read_csv(file_transactions)
df_transactions.head()

,TransactionID,Date,Time,CustomerID,CardID,GasStationID,ProductID,Amount,Price
0,1,2012-08-24,09:41:00,31543,486621,3704,2,28,672.64
1,2,2012-08-24,10:03:00,46707,550134,3704,2,18,430.72
2,3,2012-08-24,10:03:00,46707,550134,3704,23,1,121.99
3,4,2012-08-24,13:53:00,7654,684220,656,5,5,120.74
4,5,2012-08-24,08:49:00,17373,536109,741,2,28,645.05


In [8]:
df_yearmonth = pd.read_csv(file_yearmonth)
df_yearmonth.head()

,CustomerID,Date,Consumption
0,5,201207,528.30
1,5,201302,1598.28
2,5,201303,1931.36
3,5,201304,1497.14
4,6,201203,51.06


## Data Understanding

<li>Merge Tables</li>

In [ ]:
# 1. Merge transaksi dengan customers
df_merged = pd.merge(df_transactions, df_customers,
                     on='CustomerID', how='left')

# 2. Merge dengan gasstations
df_merged = pd.merge(df_merged, df_gasstations, on='GasStationID', how='left')

# 3. Merge dengan products
df_merged = pd.merge(df_merged, df_products, on='ProductID', how='left')

# 4. Siapkan kolom 'Date' dari df_transactions agar cocok dengan df_yearmonth (yyyymm)
df_merged['Date'] = pd.to_datetime(df_merged['Date'])
df_merged['YearMonth'] = df_merged['Date'].dt.strftime('%Y%m')

# 5. Ubah 'Date' di df_yearmonth jadi string
df_yearmonth['Date'] = df_yearmonth['Date'].astype(str)

# 6. Merge dengan df_yearmonth berdasarkan CustomerID dan YearMonth
df_merged = pd.merge(
    df_merged,
    df_yearmonth,
    left_on=['CustomerID', 'YearMonth'],
    right_on=['CustomerID', 'Date'],
    how='left'
)

# 7. Rapikan kolom: buang kolom Date duplikat dan kolom YearMonth
df_merged.drop(columns=['Date_y', 'YearMonth'], inplace=True)
df_merged.rename(columns={'Date_x': 'Date'}, inplace=True)

# 8. Ubah kolom 'Date' menjadi object (string)
df_merged['Date'] = df_merged['Date'].astype(str)

df_merged.head()

,TransactionID,Date,Time,CustomerID,CardID,GasStationID,ProductID,Amount,Price,Segment_x,Currency,ChainID,Country,Segment_y,Description,Consumption
0,1,2012-08-24,09:41:00,31543,486621,3704,2,28,672.64,KAM,CZK,4,CZE,Premium,Nafta,142991.10
1,2,2012-08-24,10:03:00,46707,550134,3704,2,18,430.72,KAM,CZK,4,CZE,Premium,Nafta,123653.64
2,3,2012-08-24,10:03:00,46707,550134,3704,23,1,121.99,KAM,CZK,4,CZE,Premium,Provoz.nápl.,123653.64
3,4,2012-08-24,13:53:00,7654,684220,656,5,5,120.74,KAM,CZK,30,CZE,Noname,Natural,18107.97
4,5,2012-08-24,08:49:00,17373,536109,741,2,28,645.05,KAM,CZK,2,CZE,Noname,Nafta,72690.54


<li>Data Types</li>

In [15]:
df_merged.dtypes

TransactionID      int64
Date              object
Time              object
CustomerID         int64
CardID             int64
GasStationID       int64
ProductID          int64
Amount             int64
Price            float64
Segment_x         object
Currency          object
ChainID            int64
Country           object
Segment_y         object
Description       object
Consumption      float64
dtype: object

## Soal Nomer 1

### 1. Buatlah table list Top 5 Customers yang memiliki nilai transaksi paling banyak !

In [ ]:
# Hitung total transaksi per CustomerID
total_per_customer = df_transactions.groupby(
    "CustomerID")["Price"].sum().reset_index()

# Urutkan dan ambil 5 customer teratas
top_5_customers = total_per_customer.sort_values(
    by="Price", ascending=False).head(5)

# Gabungkan dengan data pelanggan untuk informasi tambahan (segment, currency, dll)
top_5_customers = top_5_customers.merge(
    df_customers, on="CustomerID", how="left")

# Tampilkan hasil
top_5_customers

,CustomerID,Price,Segment,Currency
0,19182,8618.54,KAM,CZK
1,16358,8400.94,KAM,CZK
2,31543,6161.50,KAM,CZK
3,12459,5974.47,KAM,CZK
4,30895,5940.01,KAM,CZK


## Soal Nomer 2

### 2. Buatlah list Top 5 Gas stations yang memiliki nilai transaksi paling banyak !

In [ ]:
# Gabungkan berdasarkan GasStationID
merged = pd.merge(df_transactions, df_gasstations, on="GasStationID")

# Hitung jumlah transaksi dan total sales per GasStationID
summary = merged.groupby("GasStationID").agg(
    Amount=('Amount', 'sum'),
    Sales=('Price', 'sum')
).sort_values(by="Sales", ascending=False).head(5)

# Tampilkan dengan GasStationID sebagai indeks
summary.index.name = "Gas Stations"
summary

,Amount,Sales
Gas Stations ID,,
4347,370,8557.94
48,295,6868.67
3250,320,6744.47
1155,292,6701.69
2417,273,6065.16


## Soal Nomer 3

### 3. Buatlah list Top 5 Jenis produk yang memiliki nilai transaksi paling banyak !

In [57]:
# Gabungkan transaksi dengan data produk
merged = pd.merge(df_transactions, df_products, on="ProductID")

# Hitung total Amount dan Sales per Produk
summary = merged.groupby("Description").agg(
    Amount=("Amount", "sum"),
    Sales=("Price", "sum"),
).sort_values(by="Sales", ascending=False).head(5)

# Tampilkan hasil
summary.index.name = "Products"
summary

,Amount,Sales
Products,,
Nafta,15230,345669.45
Diesel +,1120,27261.03
Natural,1045,24639.46
Nafta Plus,317,7233.72
Natural +,157,3999.92


## Soal Nomer 4

### 4. Buatlah deskripsi statistik untuk masing masing hari (23,24,25,26) pada dataset transaction_1k !

In [22]:
# Ubah kolom Date menjadi format datetime
df_transactions["Date"] = pd.to_datetime(df_transactions["Date"])

# Filter hanya tanggal 23, 24, 25, dan 26
filtered_df = df_transactions[df_transactions["Date"].dt.day.isin([
                                                                  23, 24, 25, 26])]

# Buat deskripsi statistik per hari
daily_stats = filtered_df.groupby(filtered_df["Date"].dt.day)[
    "Price"].describe()

# Tampilkan hasil
daily_stats

,count,mean,std,min,25%,50%,75%,max
Date,,,,,,,,
23,10.0,685.379000,534.046487,76.42,481.170,619.44,795.095,1994.71
24,492.0,409.939736,488.269481,1.76,76.125,332.45,615.395,5762.49
25,425.0,426.940071,453.134207,6.09,102.220,370.38,600.850,3600.84
26,73.0,487.431370,429.192499,4.74,142.840,437.03,675.520,2303.75


In [26]:
# Ubah kolom Date menjadi format datetime
df_transactions["Date"] = pd.to_datetime(df_transactions["Date"])

# Filter hanya tanggal 23, 24, 25, dan 26
filtered_df = df_transactions[df_transactions["Date"].dt.day.isin([
                                                                  23, 24, 25, 26])]

# Buat deskripsi statistik untuk kolom Amount dan Price
summary = filtered_df.groupby("Date").agg({
    'Amount': ['count', 'mean', 'std', 'min',
               lambda x: x.quantile(0.25),
               'median',
               lambda x: x.quantile(0.75),
               'max'],
    'Price': ['count', 'mean', 'std', 'min',
              lambda x: x.quantile(0.25),
              'median',
              lambda x: x.quantile(0.75),
              'max']
})

# Rename nama fungsi lambda menjadi '25%', '50%', '75%'
summary.columns = pd.MultiIndex.from_tuples([
    (col[0],
     '25%' if i % 8 == 4 else
     '50%' if i % 8 == 5 else
     '75%' if i % 8 == 6 else col[1])
    for i, col in enumerate(summary.columns)
])

# Ubah nama kolom 'Price' menjadi 'Sales'
summary.rename(columns={"Price": "Sales"}, inplace=True)

# Tambahkan nama level kolom multi-index
summary.columns.names = ['Metric', 'Statistic']

# Tampilkan hasil
summary

Metric     Amount                                                    Sales  \
Statistic   count       mean        std min    25%   50%    75%  max count   
Date                                                                         
2012-08-23     10  29.600000  24.900692   1  21.25  26.5  35.25   91    10   
2012-08-24    492  19.936992  21.949630   0   5.00  19.0  27.00  264   492   
2012-08-25    425  18.880000  20.376114   0   4.00  17.0  26.00  154   425   
2012-08-26     73  21.219178  18.932810   0   6.00  20.0  29.00  101    73   

Metric                                                                        
Statistic         mean         std    min      25%     50%      75%      max  
Date                                                                          
2012-08-23  685.379000  534.046487  76.42  481.170  619.44  795.095  1994.71  
2012-08-24  409.939736  488.269481   1.76   76.125  332.45  615.395  5762.49  
2012-08-25  426.940071  453.134207   6.09  102.220  370.38  600.850  3600.84  
2012-08-26  487.431370  429.192499   4.74  142.840  437.03  675.520  2303.75

## Soal Nomer 5

### 5. Carilah waktu terbaik (hari dan jam) dimana paling banyak user gas station melakukan transaksi !

In [32]:
# Pastikan kolom Date dan Time dalam format string
df_transactions['Date'] = df_transactions['Date'].astype(str)
df_transactions['Time'] = df_transactions['Time'].astype(str)

# Gabungkan Date dan Time, lalu ubah ke datetime
df_transactions['Datetime'] = pd.to_datetime(
    df_transactions['Date'] + ' ' + df_transactions['Time'])

# Ekstrak nama hari dan jam dari kolom datetime
df_transactions['DayOfWeek'] = df_transactions['Datetime'].dt.day_name()
df_transactions['Hour'] = df_transactions['Datetime'].dt.hour

# Hitung jumlah transaksi per kombinasi hari dan jam
grouped = df_transactions.groupby(
    ['DayOfWeek', 'Hour']).size().reset_index(name='TransactionCount')

# Temukan kombinasi hari dan jam dengan transaksi terbanyak
best_time = grouped.sort_values(by='TransactionCount', ascending=False).head(1)

# Cetak hasil
day = best_time.iloc[0]['DayOfWeek']
hour = int(best_time.iloc[0]['Hour'])
count = int(best_time.iloc[0]['TransactionCount'])

grouped
print(
    f"Waktu terbaik dengan jumlah transaksi terbanyak adalah hari {day} jam {hour}:00 dengan {count} transaksi.")

Waktu terbaik dengan jumlah transaksi terbanyak adalah hari Friday jam 14:00 dengan 58 transaksi.


## Soal Nomer 6

### 6. Dari segi bisnis understanding, tujuan utama dari analisis data customer ini adalah? 